In [101]:
import yfinance as yf
import pandas as pd
import os
import datetime

BASE_DIR="/Users/dannyyu/Desktop/AI_Trader/data"
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'INTC', 'AMD', 'IBM']
START_DATE = "2022-01-01"
END_DATE = "2025-5-18"


ticker_obj = yf.Ticker('IBM')
info = ticker_obj.info

recent_eps = info.get("trailingEps", None)
recent_pe = info.get("trailingPE", None)


income_stmt = ticker_obj.financials  # Income statement
balance_sheet = ticker_obj.balance_sheet
cashflow_stmt = ticker_obj.cashflow

#print(income_stmt.index)
print(balance_sheet.index)
#print(cashflow_stmt.index)

Index(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Net Debt', 'Total Debt', 'Tangible Book Value', 'Invested Capital',
       'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations',
       'Common Stock Equity', 'Total Capitalization',
       'Total Equity Gross Minority Interest', 'Minority Interest',
       'Stockholders Equity', 'Other Equity Interest',
       'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Treasury Stock', 'Retained Earnings',
       'Capital Stock', 'Common Stock',
       'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Liabilities Heldfor Sale Non Current',
       'Derivative Product Liabilities', 'Employee Benefits',
       'Non Current Pension And Other Postretirement Benefit Plans',
       'Non Current Accrued Expenses', 'Tradeand Other Payables Non Current',
       'Non Current Defe

In [39]:
#income_stmt.loc["Diluted EPS", pd.Timestamp("2022-09-30")]
common_dates = income_stmt.columns.intersection(balance_sheet.columns).intersection(cashflow_stmt.columns)
print(balance_sheet.columns)

DatetimeIndex(['2024-12-31', '2023-12-31', '2022-12-31', '2021-12-31',
               '2020-12-31'],
              dtype='datetime64[ns]', freq=None)


In [5]:
common_dates = income_stmt.columns.intersection(balance_sheet.columns).intersection(cashflow_stmt.columns)
common_dates

DatetimeIndex(['2024-09-30', '2023-09-30', '2022-09-30', '2021-09-30',
               '2020-09-30'],
              dtype='datetime64[ns]', freq=None)

In [32]:
ticker=yf.Ticker("AAPL")
info = ticker.info
print(info.get("trailingPE", None))
end_date="2024-10-01"
start_date="2024-09-27"
eps=info.get("trailingEps", None)
print(eps)
hist_price = ticker.history(start=start_date, end=end_date)
print(hist_price)
hist_price=hist_price['Close'].iloc[0]
print(hist_price)

31.325039
6.43
                                 Open        High         Low       Close  \
Date                                                                        
2024-09-27 00:00:00-04:00  227.659747  228.716032  226.503807  226.992081   
2024-09-30 00:00:00-04:00  229.234200  232.183838  228.845566  232.183838   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-09-27 00:00:00-04:00  34026000        0.0           0.0  
2024-09-30 00:00:00-04:00  54541900        0.0           0.0  
226.99208068847656


In [49]:
ticker_obj=yf.Ticker("NVDA")

hist=ticker_obj.history(period="1d")#start=start_date, end=end_date)
#last_row=hist.iloc[-1]
price=hist["Close"]
price

Date
2025-05-23 00:00:00-04:00    131.289993
Name: Close, dtype: float64

In [67]:
income_stmt = ticker_obj.financials

trailing_eps=info.get("trailingPE", None)
trailing_eps

44.21026

In [69]:
trailing_pe=price/trailing_eps
trailing_pe

Date
2025-05-23 00:00:00-04:00    2.969672
Name: Close, dtype: float64

In [71]:
income_stmt.columns

DatetimeIndex(['2025-01-31', '2024-01-31', '2023-01-31', '2022-01-31'], dtype='datetime64[ns]', freq=None)

AttributeError: 'Ticker' object has no attribute 'valuation_measures'

In [103]:
def get_fundamentals_history(ticker, years=3):
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.info
    
    # Most recent static values
    recent_eps = info.get("trailingEps", None)
    recent_pe = info.get("trailingPE", None)
    
    # Annual financial statements
    income_stmt = ticker_obj.financials  # Income statement
    balance_sheet = ticker_obj.balance_sheet
    cashflow_stmt = ticker_obj.cashflow

    # All dates available (annual format)
    common_dates = income_stmt.columns.intersection(balance_sheet.columns).intersection(cashflow_stmt.columns)
    current_year = datetime.datetime.now().year
    common_dates = [date for date in common_dates if date.year != current_year]

    
    fundamentals = []
    
    #get the most recent fundamental indicators
    info = ticker_obj.info
    fundamentals.append({
                "Date": pd.Timestamp.today().strftime('%Y-%m-%d'),
                "DilutedEPS": info.get("trailingEps", None),
                "PE": info.get("trailingPE", None),
                "Revenue": info.get("totalRevenue", None),
                "CashFlow": info.get("operatingCashflow", None),
                "EBITDA": info.get("ebitda", None),
                "GrossProfit": info.get("grossProfits", None),
                "OperatingMargin": info.get("operatingMargins", None),
                "ROE": info.get("returnOnEquity", None),
                "DebtToEquity": info.get("debtToEquity", None),
            })

    #get the fundamental indicators for the past 3 years
    for date in common_dates:
        try:
            if pd.Timestamp.today().year - date.year > years:
                continue
    
        
            revenue = income_stmt.loc["Total Revenue", date]
            gross_profit = income_stmt.loc["Gross Profit", date]
            ebitda = income_stmt.loc["EBITDA", date]
            net_income = income_stmt.loc["Net Income", date]
    
            operating_cashflow = cashflow_stmt.loc["Operating Cash Flow", date]
            total_equity = balance_sheet.loc["Common Stock Equity", date]
            total_debt = balance_sheet.loc["Total Debt", date]
    
            roe = net_income / total_equity if total_equity else None
            debt_to_equity = (total_debt / total_equity)*100 if total_equity else None
            operating_margin = net_income / revenue if revenue else None

            diluted_eps = income_stmt.loc["Diluted EPS", date]

            hist_price = get_closest_price(ticker_obj, date)

            if hist_price and diluted_eps:
                pe_ratio = hist_price / diluted_eps
            else:
                pe_ratio = recent_pe
    
            fundamentals.append({
                "Date": date.strftime('%Y-%m-%d'),
                "DilutedEPS": diluted_eps,
                "PE": pe_ratio,
                "Revenue": revenue,
                "CashFlow": operating_cashflow,
                "EBITDA": ebitda,
                "GrossProfit": gross_profit,
                "OperatingMargin": round(operating_margin,8),
                "ROE": round(roe,3),
                "DebtToEquity": round(debt_to_equity,5)
            })
        except Exception as e:
            print(f"Error processing {ticker} for {date.date()}: {e}")
            continue

    return pd.DataFrame(fundamentals)

def get_closest_price(ticker_obj, target_date, lookback_days=3):
    try:
        start_date = target_date - pd.Timedelta(days=lookback_days)
        end_date = target_date + pd.Timedelta(days=1)

        hist = ticker_obj.history(start=start_date, end=end_date)
        
        if not hist.empty:
            last_row = hist.iloc[-1]
            timestamp_str = hist.index[-1].strftime("%Y-%m-%d")
           # print("Data retrieved on date "+timestamp_str)
            return last_row['Close']
        else:
            print(f"No price data found for {ticker_obj.ticker} between {start_date.date()} and {end_date.date()}")
            return None

    except Exception as e:
        print(f"Error retrieving price for {ticker_obj.ticker} near {target_date.date()}: {e}")
        return None

In [105]:
df=get_fundamentals_history("NVDA")
df

,Date,DilutedEPS,PE,Revenue,CashFlow,EBITDA,GrossProfit,OperatingMargin,ROE,DebtToEquity
0,2025-05-26,2.940,44.656460,1.304970e+11,6.408900e+10,8.331700e+10,9.785800e+10,0.611070,1.19177,12.94600
1,2024-01-31,1.190,51.683952,6.092200e+10,2.809000e+10,3.558300e+10,4.430100e+10,0.488493,0.69200,25.72479
2,2023-01-31,0.174,112.189677,2.697400e+10,5.641000e+09,5.986000e+09,1.535600e+10,0.161934,0.19800,54.43645
3,2022-01-31,0.385,63.488557,2.691400e+10,9.108000e+09,1.135100e+10,1.747500e+10,0.362339,0.36600,44.45739


In [98]:
for ticker in TICKERS:
    df=get_fundamentals_history(ticker) 
    print(ticker)
    print(df)

AAPL
         Date  DilutedEPS         PE       Revenue      CashFlow  \
0  2025-05-26        6.42  30.415888  4.003660e+11  1.095560e+11   
1  2024-09-30        6.08  38.188131  3.910350e+11  1.182540e+11   
2  2023-09-30        6.13  27.690326  3.832850e+11  1.105430e+11   
3  2022-09-30        6.11  22.292429  3.943280e+11  1.221510e+11   

         EBITDA   GrossProfit  OperatingMargin      ROE  DebtToEquity  
0  1.388660e+11  1.866990e+11         0.310290  1.38015     146.99400  
1  1.346610e+11  1.806830e+11         0.239713  1.64600     187.23266  
2  1.258200e+11  1.691480e+11         0.253062  1.56100     178.75326  
3  1.305410e+11  1.707820e+11         0.253096  1.97000     261.44616  
MSFT
         Date  DilutedEPS         PE       Revenue      CashFlow  \
0  2025-05-26       12.92  34.843650  2.700100e+11  1.307100e+11   
1  2024-06-30       11.80  37.588928  2.451220e+11  1.185480e+11   
2  2023-06-30        9.68  34.641096  2.119150e+11  8.758200e+10   
3  2022-06-30    

         Date  DilutedEPS          PE       Revenue      CashFlow  \
0  2025-05-26       2.940   44.656460  1.304970e+11  6.408900e+10   
1  2025-01-31       2.940   40.836381  1.304970e+11  6.408900e+10   
2  2024-01-31       1.190   51.683952  6.092200e+10  2.809000e+10   
3  2023-01-31       0.174  112.189677  2.697400e+10  5.641000e+09   
4  2022-01-31       0.385   63.488557  2.691400e+10  9.108000e+09   

         EBITDA   GrossProfit  OperatingMargin      ROE  DebtToEquity  
0  8.331700e+10  9.785800e+10         0.611070  1.19177      12.94600  
1  8.613700e+10  9.785800e+10         0.558480  0.91900      12.94641  
2  3.558300e+10  4.430100e+10         0.488493  0.69200      25.72479  
3  5.986000e+09  1.535600e+10         0.161934  0.19800      54.43645  
4  1.135100e+10  1.747500e+10         0.362339  0.36600      44.45739  
